In [1]:
import re
import numpy as np
import sys
from pathlib import Path

# sys.path.append(str(Path("/home/yann/ssd_storage/python/arcprize2025/tests/")))
# sys.path.append(str(Path("/home/yann/ssd_storage/python/arcprize2025/sources/")))

sys.path.append(str(Path("/workspaces/GridGlyph/sources")))
sys.path.append(str(Path("/workspaces/GridGlyph/tests")))

from test_dsl_symbolic_executor import TEST_CASES
from core.dsl_symbolic_interpreter import SYMBOL_RULES
from utils.synthetic_grids import (
        generate_single_random_grid,
        create_base_grid,
        generate_random_shape_grid,
        )

In [2]:
ATOMIC_PATTERNS = []
CONDITIONALLY_ATOMIC_PATTERNS_CHECKERS = {}
NON_ATOMIC_PATTERNS = []

for rule_key, rule_definition in SYMBOL_RULES.items():
    sigil = rule_definition.get("sigil")
    pattern = rule_definition.get("pattern")
    nested_commands = rule_definition.get("nested_commands")

    if sigil and not pattern and not nested_commands:
        ATOMIC_PATTERNS.append(re.compile(rf"^{re.escape(sigil)}$"))
        continue

    if not pattern:
        continue

    try:
        compiled_pattern = re.compile(pattern)
    except re.error:
        continue

    if nested_commands is None or nested_commands == {}:
        ATOMIC_PATTERNS.append(compiled_pattern)
    elif rule_key in ["flip_h", "flip_v", "flatten_grid", "extract_bounding_box", "reverse_row"]:
        def make_checker(rule_key_inner):
            def checker(match):
                arg_content = match.group("arg_content") if match.lastindex and match.lastgroup == "arg_content" else None
                return arg_content is None or arg_content.strip() == "⌂"
            return checker
        CONDITIONALLY_ATOMIC_PATTERNS_CHECKERS[compiled_pattern] = make_checker(rule_key)
    else:
        NON_ATOMIC_PATTERNS.append(compiled_pattern)

block_builder_def = SYMBOL_RULES.get("block_grid_builder")
if block_builder_def and "pattern" in block_builder_def:
    try:
        ATOMIC_PATTERNS.append(re.compile(block_builder_def["pattern"]))
    except re.error:
        pass

In [3]:
def is_atomic_rule(rule_str: str) -> bool:
    rule_str = rule_str.strip()
    
    # --- Special case for the match_pattern symbol (◫) ---
    if rule_str.startswith('◫('):
        return False
    # --- End of special case ---
    
    for non_atomic_pattern in NON_ATOMIC_PATTERNS:
        if non_atomic_pattern.match(rule_str):
            return False
            
    for atomic_pattern in ATOMIC_PATTERNS:
        if atomic_pattern.match(rule_str):
            return True
            
    for conditional_pattern, checker in CONDITIONALLY_ATOMIC_PATTERNS_CHECKERS.items():
        match = conditional_pattern.match(rule_str)
        if match and checker(match):
            return True
            
    return False

In [4]:
atomic_training_data = []

for rule_str, input_grid_np, _ in TEST_CASES:
    if not is_atomic_rule(rule_str):
        continue

    if input_grid_np is not None:
        input_grid_list = input_grid_np.tolist()
    else:
        if "▦(" in rule_str:
             input_grid_list = []
        else:
             continue

    atomic_training_data.append({
        "input_grid": input_grid_list,
        "dsl_rule": rule_str
    })

print(f"Found {len(atomic_training_data)} atomic training examples.")


Found 110 atomic training examples.


In [5]:
import numpy as np
import sys
from pathlib import Path

def execute_dsl_rule_on_grid(input_grid_list: list[list[int]], dsl_rule_str: str) -> np.ndarray:
    
    current_dir = Path.cwd()
    project_root_candidates = [
        current_dir,
        current_dir.parent,
        current_dir.parent.parent
    ]
    
    src_path = None
    for root_candidate in project_root_candidates:
        if (root_candidate / "sources").exists():
            src_path = root_candidate / "sources"
            break
            
    if src_path is None:
        raise FileNotFoundError("Could not find the 'sources' directory. Please ensure your working directory is correctly set relative to 'sources' or manually adjust the 'src_path' variable in this function.")

    if str(src_path) not in sys.path:
        sys.path.append(str(src_path))

    try:
        from core.dsl_symbolic_interpreter import SymbolicRuleParser
        from core.dsl_symbolic_executor import DSLExecutor
    except ImportError as e:
        raise ImportError(f"Failed to import DSL components. Ensure 'core/dsl_symbolic_interpreter.py' and 'core/dsl_symbolic_executor.py' exist in your '{src_path}' directory. Error: {e}")

    input_grid_np = np.array(input_grid_list, dtype=int)
    parser = SymbolicRuleParser()

    try:
        command = parser.parse_rule(dsl_rule_str)
        executor = DSLExecutor(
            root_command=command,
            initial_puzzle_input=input_grid_np,
        )
        result_grid_np = executor.execute_program()
        return result_grid_np
    except Exception as e:
        print(f"Error executing rule '{dsl_rule_str}' on input grid: {e}")
        return np.array([]) 

In [6]:
import numpy as np
import random

def int_to_roman(num: int) -> str:
    mapping = {
        0: "∅", 1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V',
        6: 'VI', 7: 'VII', 8: 'VIII', 9: 'IX', 10: 'X',
        11: 'XI', 12: 'XII', 13: 'XIII', 14: 'XIV', 15: 'XV',
        16: 'XVI', 17: 'XVII', 18: 'XVIII', 19: 'XIX', 20: 'XX',
        21: 'XXI', 22: 'XXII', 23: 'XXIII', 24: 'XXIV', 25: 'XXV',
        26: 'XXVI', 27: 'XXVII', 28: 'XXVIII', 29: 'XXIX', 30: 'XXX'
    }
    return mapping[num]

        
def roman_to_int(roman_numeral: str) -> int:
    mapping = {
        '∅': 0, # Add zero mapping
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5,
        'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10,
        'XI': 11, 'XII': 12, 'XIII': 13, 'XIV': 14, 'XV': 15,
        'XVI': 16, 'XVII': 17, 'XVIII': 18, 'XIX': 19, 'XX': 20,
        'XXI': 21, 'XXII': 22, 'XXIII': 23, 'XXIV': 24, 'XXV': 25,
        'XXVI': 26, 'XXVII': 27, 'XXVIII': 28, 'XXIX': 29, 'XXX': 30
    }
    return mapping.get(roman_numeral.upper(), None)


def is_rule_compatible_with_grid(dsl_rule_str: str, grid_shape: tuple) -> bool:
    rows, cols = grid_shape

    if rows < 1 or cols < 1:
        return False

    if dsl_rule_str.startswith('⇅('):
        match = re.match(r'⇅\(([IVX]+),([IVX]+)\)', dsl_rule_str)
        if not match:
            print(f"Warning: Malformed ⇅ rule string encountered: {dsl_rule_str}")
            return False
        
        row_str1, row_str2 = match.groups()
        idx1 = roman_to_int(row_str1)
        idx2 = roman_to_int(row_str2)

        if idx1 > 0 and idx2 > 0 and idx1 <= rows and idx2 <= rows:
            return True
        else:
            return False
        
    return True

def mutate_flip_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    mutated_items = []
    original_grid = np.array(item['input_grid'], dtype=int)
    original_rule = item['dsl_rule']

    for _ in range(num_variants - 1):
        new_grid = generate_single_random_grid(original_grid.shape, max_dim, num_range)
        mutated_items.append({'input_grid': new_grid.tolist(), 'dsl_rule': original_rule})

    return mutated_items

def mutate_swap_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9, is_row_swap: bool = True) -> list[dict]:
    mutated_items = []
    original_rule = item['dsl_rule']
    pattern = r'⇅\((?P<idx1>[IVX]+),(?P<idx2>[IVX]+)\)' if is_row_swap else r'⇄\((?P<idx1>[IVX]+),(?P<idx2>[IVX]+)\)'
    sigil = '⇅' if is_row_swap else '⇄'
    
    match = re.match(pattern, original_rule)
    if not match:
        print(f"Warning: Could not parse swap rule: {original_rule}")
        return []
    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 100
    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        dim_size = candidate_grid_np.shape[0] if is_row_swap else candidate_grid_np.shape[1]
        if dim_size >= 2:
            new_idx1_idx = random.randint(1, dim_size)
            new_idx2_idx = random.randint(1, dim_size)
            while new_idx2_idx == new_idx1_idx:
                new_idx2_idx = random.randint(1, dim_size)
            new_idx1_name = int_to_roman(new_idx1_idx)
            new_idx2_name = int_to_roman(new_idx2_idx)
            new_rule = f"{sigil}({new_idx1_name},{new_idx2_name})"
            mutated_items.append({
                'input_grid': candidate_grid_np.tolist(),
                'dsl_rule': new_rule
            })
        attempts += 1
    if len(mutated_items) < target_count:
        print(f"Warning: mutate_swap_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")
    return mutated_items

def mutate_swap_row_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    return mutate_swap_rule(item, num_variants, max_dim, num_range, is_row_swap=True)

def mutate_swap_col_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    return mutate_swap_rule(item, num_variants, max_dim, num_range, is_row_swap=False)

def mutate_swap_value_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    """
    Mutates a value swap rule (⇒(A, B)) by generating new grids and updating the values in the rule.
    Ensures the 'from' value (A) exists in the new grid.
    """
    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'⇒\((?P<from_val>[^,]*),\s*(?P<to_val>[^)]*)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse swap value rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 100

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        # 1. Generate a new candidate grid
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        
        # 2. Determine new values for the rule (0 to num_range)
        new_from_val_int = random.randint(0, num_range)
        new_to_val_int = random.randint(0, num_range)
        
        # Convert integers to DSL symbols using the updated int_to_roman
        new_from_val_sym = int_to_roman(new_from_val_int)
        new_to_val_sym = int_to_roman(new_to_val_int)

        # 3. Create the new rule string
        new_rule = f"⇒({new_from_val_sym}, {new_to_val_sym})"

        # 4. Ensure the 'from' value exists in the grid, or inject it
        if new_from_val_int in candidate_grid_np:
            # Value already exists, grid is fine
            final_grid = candidate_grid_np
        else:
            # Need to inject the 'from' value to make the rule meaningful
            # Modify one random element to be the 'from' value
            if candidate_grid_np.size > 0:
                flat_index = random.randint(0, candidate_grid_np.size - 1)
                final_grid = candidate_grid_np.copy()
                final_grid.flat[flat_index] = new_from_val_int
            else:
                final_grid = candidate_grid_np # Keep empty grid if size is 0

        mutated_items.append({
            'input_grid': final_grid.tolist(),
            'dsl_rule': new_rule
        })
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_swap_value_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

def mutate_extract_value_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    """
    Mutates an extract value rule (⊡(R,C)) by generating new grids and updating the row/column indices.
    Ensures the new indices are within the bounds of the new grid.
    """
    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'⊡\((?P<row>[IVX∅]+),(?P<col>[IVX∅]+)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse extract value rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 100

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        # 1. Generate a new candidate grid
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        
        rows, cols = candidate_grid_np.shape
        
        # Need at least a 1x1 grid to extract a value
        if rows >= 1 and cols >= 1:
            # 2. Select valid 1-based indices for the new grid
            new_row_idx = random.randint(1, rows)
            new_col_idx = random.randint(1, cols)

            # 3. Convert indices to DSL symbols
            new_row_sym = int_to_roman(new_row_idx)
            new_col_sym = int_to_roman(new_col_idx)

            # 4. Create the new rule string
            new_rule = f"⊡({new_row_sym},{new_col_sym})"

            mutated_items.append({
                'input_grid': candidate_grid_np.tolist(),
                'dsl_rule': new_rule
            })
            
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_extract_value_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

def mutate_identity_rule(item: dict, num_variants: int = 3, max_dim: int = 30, num_range: int = 9) -> list[dict]:
    """
    Mutates an identity rule (⌂) by generating new random grids.
    The rule string '⌂' remains unchanged.
    """
    mutated_items = []
    original_rule = item['dsl_rule'] # This should be '⌂'
    
    for _ in range(num_variants - 1):
        # 1. Generate a new candidate grid
        candidate_grid_np = generate_single_random_grid(np.array(item['input_grid']).shape, max_dim, num_range)
        
        # 2. Append the new grid with the unchanged rule
        mutated_items.append({
            'input_grid': candidate_grid_np.tolist(),
            'dsl_rule': original_rule # Remains '⌂'
        })
        
    return mutated_items


def mutate_extract_background_rule(item: dict, num_variants: int , max_dim: int = 30, num_range: int = 9) -> list[dict]:
    mutated_items = []
    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 150

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        try:
            # 1. Decide on the new background value (V_new) for this grid/rule
            new_background_val_int = random.randint(0, num_range) # V can be 0 (null/empty) or 1-9
            new_background_val_sym = int_to_roman(new_background_val_int)
            new_rule = f"⏚({new_background_val_sym})"

            # 2. Generate a grid using this value as the background
            candidate_grid_np = generate_random_shape_grid(
                min_dim=3,
                max_dim=max_dim,
                value_range=(0, max(0, num_range)), # Shapes can use 0 or other values
                num_shapes=random.randint(1, 5),
                fill_prob=random.uniform(0.2, 0.5),
                background_value=new_background_val_int # Crucial: Set the grid's background to V_new
            )

            # 3. Basic validation: need a non-empty grid
            if candidate_grid_np.size >= 1:
                # 4. Append the valid example
                mutated_items.append({
                    'input_grid': candidate_grid_np.tolist(),
                    'dsl_rule': new_rule
                })

        except Exception as e:
            # Silently handle generation errors for robustness
            pass
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_extract_background_rule could not generate {target_count} valid mutations. Got {len(mutated_items)}.")

    return mutated_items


def mutate_extract_value_occurrences_rule(item: dict, num_variants: int = 5, max_dim: int = 20, num_range: int = 9) -> list[dict]:
    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'◎\((?P<value>[^)]*)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse extract value occurrences rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 150

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        try:
            orig_shape = np.array(item['input_grid']).shape
            candidate_grid_np = generate_single_random_grid(orig_shape, max_dim, num_range)

            if candidate_grid_np.size >= 1:
                new_target_val_int = random.randint(0, num_range)
                new_target_val_sym = int_to_roman(new_target_val_int)
                new_rule = f"◎({new_target_val_sym})"

                final_grid = candidate_grid_np.copy()
                # Strategy: Always ensure at least one instance of the target value exists.
                # This makes the rule non-trivial.
                if new_target_val_int not in candidate_grid_np:
                    # If not present, inject it.
                    flat_index = random.randint(0, candidate_grid_np.size - 1)
                    final_grid.flat[flat_index] = new_target_val_int
                else:
                    # If present, optionally add one more instance to ensure variety
                    # or make the pattern slightly more interesting.
                    # This is a simple tweak, can be more sophisticated.
                    if random.random() < 0.3 and candidate_grid_np.size > 1: # 30% chance to add another if already exists
                         flat_index = random.randint(0, candidate_grid_np.size - 1)
                         # Avoid overwriting an existing target val if possible (simple check)
                         if final_grid.flat[flat_index] != new_target_val_int:
                              final_grid.flat[flat_index] = new_target_val_int
                         # If it was the same, we just leave it.

                mutated_items.append({
                    'input_grid': final_grid.tolist(),
                    'dsl_rule': new_rule
                })

        except Exception:
            pass
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_extract_value_occurrences_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

def mutate_get_connected_component_rule(item: dict, num_variants: int = 5, max_dim: int = 20, num_range: int = 9) -> list[dict]:
    from utils.synthetic_grids import generate_random_shape_grid

    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'⚇\((?P<row>[^,]+),(?P<col>[^)]+)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse get connected component rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 150

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        try:
            # Use the shape-based generator with full value range
            orig_shape = np.array(item['input_grid']).shape
            if orig_shape and len(orig_shape) == 2:
                min_dim_gen = max(2, orig_shape[0] - 2)
                # Increase the max_dim for larger grids
                effective_max_dim = min(30, max(min_dim_gen + 8, 20)) # Cap at 30 for sanity
            else:
                min_dim_gen = 2
                effective_max_dim = 20

            # Generate a grid with shapes using the full num_range
            candidate_grid_np = generate_random_shape_grid(
                min_dim=min_dim_gen,
                max_dim=effective_max_dim,
                value_range=(1, num_range), # Use the full range, including higher numbers
                num_shapes=random.randint(1, 4),
                fill_prob=0.8,
                background_value=0
            )

            if candidate_grid_np.size >= 1:
                rows, cols = candidate_grid_np.shape

                # Find a suitable starting position
                found_position = False
                for _ in range(10):
                    r_idx = random.randint(0, rows - 1)
                    c_idx = random.randint(0, cols - 1)
                    if candidate_grid_np[r_idx, c_idx] != 0:
                        neighbors = [(r_idx-1,c_idx), (r_idx+1,c_idx), (r_idx,c_idx-1), (r_idx,c_idx+1)]
                        for nr, nc in neighbors:
                            if 0 <= nr < rows and 0 <= nc < cols:
                                if candidate_grid_np[nr, nc] == candidate_grid_np[r_idx, c_idx]:
                                    found_position = True
                                    break
                        if found_position:
                            break

                if not found_position:
                    non_zero_positions = np.where(candidate_grid_np != 0)
                    if non_zero_positions[0].size > 0:
                        idx = random.randint(0, len(non_zero_positions[0]) - 1)
                        r_idx, c_idx = non_zero_positions[0][idx], non_zero_positions[1][idx]
                    else:
                        r_idx, c_idx = 0, 0

                new_row_idx = r_idx + 1
                new_col_idx = c_idx + 1

                new_row_sym = int_to_roman(new_row_idx)
                new_col_sym = int_to_roman(new_col_idx)
                new_rule = f"⚇({new_row_sym},{new_col_sym})"

                mutated_items.append({
                    'input_grid': candidate_grid_np.tolist(),
                    'dsl_rule': new_rule
                })

        except Exception as e:
            pass
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_get_connected_component_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

def mutate_crop_rule(item: dict, num_variants: int = 5, max_dim: int = 20, num_range: int = 9) -> list[dict]:
    mutated_items = []
    original_rule = item['dsl_rule']
    match = re.match(r'✂\((?P<r1>[^,]+),(?P<c1>[^,]+),(?P<r2>[^,]+),(?P<c2>[^)]+)\)', original_rule)
    if not match:
        print(f"Warning: Could not parse crop rule: {original_rule}")
        return []

    target_count = num_variants - 1
    attempts = 0
    max_attempts_per_valid_variant = 150

    while len(mutated_items) < target_count and attempts < max_attempts_per_valid_variant * target_count:
        try:
            orig_shape = np.array(item['input_grid']).shape
            candidate_grid_np = generate_single_random_grid(orig_shape, max_dim, num_range)

            if candidate_grid_np.size >= 1:
                rows, cols = candidate_grid_np.shape

                # Ensure the region is valid (at least 1x1)
                if rows < 2 or cols < 2:
                    continue

                # Randomly select a subgrid region
                r1 = random.randint(1, rows)
                c1 = random.randint(1, cols)
                r2 = random.randint(r1, rows)
                c2 = random.randint(c1, cols)

                # Convert to DSL symbols (Roman numerals)
                r1_sym = int_to_roman(r1)
                c1_sym = int_to_roman(c1)
                r2_sym = int_to_roman(r2)
                c2_sym = int_to_roman(c2)

                new_rule = f"✂({r1_sym},{c1_sym},{r2_sym},{c2_sym})"

                mutated_items.append({
                    'input_grid': candidate_grid_np.tolist(),
                    'dsl_rule': new_rule
                })

        except Exception:
            pass
        attempts += 1

    if len(mutated_items) < target_count:
        print(f"Warning: mutate_crop_rule could not generate {target_count} valid mutations for rule '{original_rule}'. Got {len(mutated_items)} after {attempts} attempts.")

    return mutated_items

In [7]:
# 1. Sort the list by the first character of the 'dsl_rule'
atomic_training_data.sort(key=lambda item: item['dsl_rule'][0])

# 2. Group the sorted list by the first character of the 'dsl_rule'
from itertools import groupby
grouped_data = groupby(atomic_training_data, key=lambda item: item['dsl_rule'][0])
groups = {key: list(group) for key, group in grouped_data}

In [8]:
outputed_dataset = []

In [9]:
mutation_functions_map = {
    # '↢': (lambda item, num_variants, **kwargs: [], 2)
    # '↔': [mutate_flip_rule, 2],
    # '↕': [mutate_flip_rule, 2],
    # '⇅': [mutate_swap_row_rule, 2],
    # '⇄': [mutate_swap_col_rule, 2],
    # '⇒': [mutate_swap_value_rule, 2],
    # '⊡': [mutate_extract_value_rule, 3],
    # '⌂': [mutate_identity_rule, 3] ,
    # '⏚': (mutate_extract_background_rule, 2),
    # '◎': (mutate_extract_value_occurrences_rule, 2),
    # '⚇': (mutate_get_connected_component_rule, 3)
    '✂': (mutate_crop_rule, 2)
}

for dsl_symbol, initial_rules_list in groups.items():
    mutate_function , num_variants_per_rule = mutation_functions_map.get(dsl_symbol, (None, None))

    if mutate_function is None:
        print(f"Warning: No mutation function defined for DSL symbol '{dsl_symbol}'. Skipping this group.")
        continue

    for rule_input_pair in initial_rules_list:
        mutated_rules: list = mutate_function(rule_input_pair, num_variants=num_variants_per_rule)
    
        rule_input_pair["output_grid"] = execute_dsl_rule_on_grid(
                input_grid_list=rule_input_pair['input_grid'],
                dsl_rule_str=rule_input_pair['dsl_rule']
            ).tolist()
        outputed_dataset.append(rule_input_pair)
        
        for mutated_pair in mutated_rules:
            mutated_pair["output_grid"] = execute_dsl_rule_on_grid(
                input_grid_list=mutated_pair['input_grid'],
                dsl_rule_str=mutated_pair['dsl_rule']
            ).tolist()
            outputed_dataset.append(mutated_pair)

2025-08-01 14:32:43,110 - core.dsl_symbolic_executor - INFO - Executor initialization started.
2025-08-01 14:32:43,110 - core.dsl_symbolic_executor - INFO - Executor initialization complete.
2025-08-01 14:32:43,111 - core.dsl_symbolic_executor - INFO - Starting DSL program execution.
2025-08-01 14:32:43,112 - core.dsl_symbolic_executor - INFO - DSL program execution completed successfully.
2025-08-01 14:32:43,113 - core.dsl_symbolic_executor - INFO - Executor initialization started.
2025-08-01 14:32:43,115 - core.dsl_symbolic_executor - INFO - Executor initialization complete.
2025-08-01 14:32:43,115 - core.dsl_symbolic_executor - INFO - Starting DSL program execution.
2025-08-01 14:32:43,116 - core.dsl_symbolic_executor - INFO - DSL program execution completed successfully.
2025-08-01 14:32:43,118 - core.dsl_symbolic_executor - INFO - Executor initialization started.
2025-08-01 14:32:43,120 - core.dsl_symbolic_executor - INFO - Executor initialization complete.
2025-08-01 14:32:43,121

In [10]:
import json
from pprint import pprint
# print(json.dumps(outputed_dataset, indent=4))
pprint(outputed_dataset, width=500)

[{'dsl_rule': '✂(I,I,II,II)', 'input_grid': [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 'output_grid': [[1, 2], [4, 5]]},
 {'dsl_rule': '✂(I,V,VII,VIII)', 'input_grid': [[2, 6, 4, 9, 7, 8, 3, 2, 7, 5], [2, 9, 8, 5, 5, 1, 1, 1, 1, 5], [3, 5, 7, 3, 1, 5, 4, 3, 9, 7], [4, 4, 1, 2, 3, 4, 4, 9, 3, 7], [5, 1, 5, 2, 3, 2, 4, 2, 1, 1], [8, 6, 7, 2, 3, 1, 2, 8, 1, 8], [7, 4, 4, 1, 4, 4, 7, 9, 1, 6], [3, 8, 3, 4, 5, 8, 1, 2, 2, 7], [9, 1, 4, 1, 3, 7, 9, 1, 1, 4]], 'output_grid': [[7, 8, 3, 2], [5, 1, 1, 1], [1, 5, 4, 3], [3, 4, 4, 9], [3, 2, 4, 2], [3, 1, 2, 8], [4, 4, 7, 9]]},
 {'dsl_rule': '✂(I,II,III,II)', 'input_grid': [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 'output_grid': [[2], [5], [8]]},
 {'dsl_rule': '✂(III,II,III,II)', 'input_grid': [[4, 2, 7], [7, 3, 5], [4, 6, 3]], 'output_grid': [[6]]},
 {'dsl_rule': '✂(I,I,III,III)', 'input_grid': [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 'output_grid': [[1, 2, 3], [4, 5, 6], [7, 8, 9]]},
 {'dsl_rule': '✂(I,II,II,III)', 'input_grid': [[6, 6, 4], [1, 1, 6]], 'output_grid

In [11]:
groups_list = list(groups.items())
groups_list[10]

('⚇',
 [{'input_grid': [[1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 0]],
   'dsl_rule': '⚇(I, I)'},
  {'input_grid': [[1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 0]],
   'dsl_rule': '⚇(I, I)'},
  {'input_grid': [[0, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]],
   'dsl_rule': '⚇(II, II)'},
  {'input_grid': [[1, 1, 0, 2, 2], [1, 0, 0, 2, 0], [0, 0, 3, 3, 3]],
   'dsl_rule': '⚇(I, IV)'},
  {'input_grid': [[0, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]],
   'dsl_rule': '⚇(I, III)'},
  {'input_grid': [[0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [1, 1, 1, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0]],
   'dsl_rule': '⚇(II, II)'},
  {'input_grid': [[1, 1, 0, 1, 0], [1, 1, 0, 0, 0], [0, 0, 0, 0, 0]],
   'dsl_rule': '⚇(I, I)'},
  {'input_grid': [[0, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0, 0, 1]],
   'dsl_rule': '⚇(II, II)'},
  {'input_grid': [[1, 1, 0, 2, 2], [1, 0, 0, 2, 0], [0, 0, 3, 3, 3]],
   'dsl_rule': '⚇(I, IV)'},
  {'input_grid': [[0, 1, 0, 0, 0], [0, 1, 1, 0, 1], [0, 0, 0,

In [12]:
len(groups_list)

18

In [13]:
# groups['↕']